In [1]:
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langchain_core.documents import Document
from langchain_core.messages import SystemMessage, HumanMessage, AnyMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
from typing import Annotated, TypedDict, Optional, Literal
import json
import logging
import os
from dotenv import load_dotenv
import datetime

from pydantic import Field

from backend.app.client import tools, book_appointment, check_availability


ModuleNotFoundError: No module named 'backend'

In [ ]:
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
model = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=GROQ_API_KEY
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

#langgraph state
class AgentState(TypedDict):
    extracted_data: dict
    messages: Annotated[list[AnyMessage], add_messages]


In [ ]:
res = model.invoke("Hello, world!")
print(res)


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


content="Hello, world. It's nice to meet you. Is there something I can help you with, or would you like to chat?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 39, 'total_tokens': 67, 'completion_time': 0.041796999, 'completion_tokens_details': None, 'prompt_time': 0.009322904, 'prompt_tokens_details': None, 'queue_time': 0.055661215, 'total_time': 0.051119903}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bd5d0-e9d8-72d2-a955-025ae243f255-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 39, 'output_tokens': 28, 'total_tokens': 67}


In [ ]:

PROMPT = """ You are a professional appoitment and reporting assistant. 
Your task is to chat with customers, understand their needs regarding medical appointments,
and assist them in booking appointment, checking availablity using the tools provided.

If information is missing for tool calling ask the user for missing information.

Once the task is done stop the conversation by thanking the user. Don't call any tool after that.
"""


In [ ]:
from langchain_groq.chat_models import ChatGroq
from typing import TypedDict, Annotated, Optional
from langgraph.graph.message import add_messages
from langchain_core.messages import SystemMessage, HumanMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode


import os
from dotenv import load_dotenv

from app.client import tools
# from utils.utils import logger
from backend.utils import get_today_date


PROMPT = """ You are a professional appoitment and reporting assistant. 
Your task is to chat with customers, understand their needs regarding medical appointments,
and assist them in booking appointment, checking availablity using the tools provided.

If information is missing for tool calling ask the user for missing information.

Once the task is done stop the conversation by thanking the user. Don't call any tool after that.
"""


class AgentState(TypedDict):
    human_input: str
    messages: Annotated[list[AnyMessage], add_messages]


def agent_with_tools(state: AgentState):
    llm_with_tools = model.bind_tools(tools)

    return {
        "messages": [llm_with_tools.invoke(state["messages"])],
    }
  
def build_prompt(state):
    messages = state["messages"].copy()
    messages.insert(0, SystemMessage(content=PROMPT.strip()))
    return {"messages": messages}

def should_continue(state: AgentState) -> str:
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "continue"
    return "end"

def build_agent_graph() -> StateGraph:
    graph = StateGraph(AgentState)
    
    # Streamlined Nodes
    graph.add_node("agent", agent_with_tools)
    graph.add_node("build_prompt", build_prompt)
    graph.add_node("tools", ToolNode(tools))

    graph.add_edge(START, "build_prompt")
    graph.add_edge("build_prompt", "agent")
    graph.add_conditional_edges(
        "agent", 
        should_continue, 
        {
            "continue": "tools", 
            "end": END
        }
    )
    graph.add_edge("tools", "agent")

    # logger.info("Agent graph successfully built.")
    return graph.compile()

In [1]:
r = build_agent_graph()
r
question = "I would like to book an appointment with Dr. Smith today at 3 PM."
state: AgentState = {"messages": [HumanMessage(content=question)]}
res = r.invoke(state)

NameError: name 'build_agent_graph' is not defined

In [2]:
from langchain_groq.chat_models import ChatGroq
from typing import TypedDict, Annotated, Optional
from langgraph.graph.message import add_messages
from langchain_core.messages import SystemMessage, HumanMessage, AnyMessage, ToolMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

model = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=GROQ_API_KEY,
    temperature=0
)


from app.client import tools
# from utils.utils import logger
from backend.utils import get_today_date

PROMPT = """ 
### Role
You are a highly efficient Medical Appointment and Reporting Assistant. Your goal is to guide users through the scheduling process with clinical precision and professional warmth.

### Operational Workflow (Graph Logic)
1. Greet: Start by acknowledging the user.
2. Context Retrieval: Check the conversation history for Doctor names, Dates, and Times.
3. Validation:
   - If a specific Doctor/Date/Time is missing, ASK for it.
   - For dates/times, assume the current year is 2026. 
   - FORMAT: Always interpret and pass dates to tools in 'YYYY-MM-DD HH:MM' format.
4. Tool Execution: Once all parameters (e.g., doctor_name, appointment_time) are clear, invoke the tool. 
5. Reporting: Translate the tool's raw output (e.g., success/failure) into a clear message for the user.

### Constraints & Guardrails
- Data Format: When a user says "Tomorrow at 10 AM," calculate the exact date based on the current date (Tuesday, Jan 20, 2026) and format it as '2026-01-21 10:00'.
- One Task at a Time: Do not call multiple tools simultaneously unless necessary.
- Finality: After a successful booking or report delivery, thank the user and signal the end of the transaction. Do not suggest further tools unless prompted.
"""


class AgentState(TypedDict):
    human_input: str
    messages: Annotated[list[AnyMessage], add_messages]

# Since the mcp is asynchronous, we need to make the agent node async as well. [This was the cause of previous errors.]
async def agent_with_tools(state: AgentState):
    llm_with_tools = model.bind_tools(tools)
    response = await llm_with_tools.ainvoke(state["messages"])
    return {
        "messages": [response]
    }

def handle_tool_error(state):
    """
    This node intercepts the output from the 'tools' node. 
    If an error occurred, it formats a message telling the agent what happened.
    """
    last_message = state["messages"][-1]
    if "error" in last_message.content.lower():
        return {
            "messages": [
                ToolMessage(
                    tool_call_id=last_message.tool_call_id,
                    content=f"Error: {last_message.content}. Please explain the error in human readable form and dont try to call the tool again."
                )
            ]
        }
    return state
  
# def build_prompt(state):
#     messages = state["messages"].copy()
#     messages.insert(0, SystemMessage(content=PROMPT.strip()+ "\n\n### Today's Date: " + get_today_date()))
#     return {"messages": messages}

# def should_continue(state: AgentState) -> str:
#     last_message = state["messages"][-1]
#     if hasattr(last_message, "tool_calls") and last_message.tool_calls:
#         return "continue"
#     return "end"

def build_agent_graph() -> StateGraph:
    graph = StateGraph(AgentState)
    
    # Streamlined Nodes
    graph.add_node("agent", agent_with_tools)
    graph.add_node("error_handler", handle_tool_error)
    graph.add_node("tools", ToolNode(tools))

    graph.add_edge(START, "agent")
    graph.add_conditional_edges(
        "agent", 
        tools_condition
    )
    graph.add_edge("tools", "error_handler")
    graph.add_edge("error_handler", "agent")
    # graph.add_edge('agent', END)

    # logger.info("Agent graph successfully built.")
    return graph.compile()

ModuleNotFoundError: No module named 'app.client'; 'app' is not a package

In [3]:
graph = build_agent_graph()
graph

NameError: name 'build_agent_graph' is not defined

In [4]:
import asyncio
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage

async def run_agent(user_query: str):
    # 1. Compile the graph
    app = build_agent_graph()
    
    # 2. Prepare the initial state
    # We include the SystemMessage and the HumanMessage here 
    # so they are the foundation of the 'messages' list.
    sys_msg = SystemMessage(content=f"{PROMPT.strip()}\n\n### Today's Date: {get_today_date()}")
    human_msg = HumanMessage(content=user_query)
    
    inputs = {"messages": [sys_msg, human_msg]}
    
    # 3. Iterate through the stream
    # Each 'event' is a dict like: {'node_name': {'messages': [...]}}
    print(f"--- Starting Agent for query: {user_query} ---")
    
    async for event in app.astream(inputs, stream_mode="updates"):
        for node_name, output in event.items():
            print(f"\n[Node: {node_name}]")
            
            # Access the messages added in this step
            if "messages" in output:
                new_message = output["messages"][-1]
                
                # Pretty print based on message type
                if hasattr(new_message, "tool_calls") and new_message.tool_calls:
                    for tc in new_message.tool_calls:
                        print(f"  🛠️  Tool Call: {tc['name']}({tc['args']})")
                elif isinstance(new_message, ToolMessage):
                    print(f"  ✅ Tool Result: {new_message.content}")
                else:
                    print(f"  🤖 Response: {new_message.content}")

    print("\n--- Interaction Complete ---")

# To run the script
def main():
    try:
        # Check if an event loop is already running (e.g., in Jupyter)
        loop = asyncio.get_running_loop()
        loop.create_task(run_agent("Is Dr. Smith available on Feb 15th?"))
    except RuntimeError:
        # No running loop, use asyncio.run
        asyncio.run(run_agent("Is Dr. Smith available on Feb 15th?"))


In [5]:
main()